In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
# Enable inline plotting  
%matplotlib inline

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
cores_per_node_comet = {
                  '1': [1, 4, 8, 16, 24],
                  '2': [24, 36, 48],
                  '4': [72, 84, 96],
                  '6': [120, 132, 144],
                  '8': [168, 180, 192],
                  '12': [264, 276, 288],
                  '16': [362, 372, 384],
                  '20': [456, 468, 480],
                  '24': [552, 564, 576],
                  '28': [648, 660, 672],
                  '32': [744, 756, 768]
                  }

cores_per_node_redo = {
                  '1': [1, 4, 8, 16, 24]
                  }

comet_sum = sum([len(i) for i in cores_per_node_comet.values()])
redo_sum = sum([len(i) for i in cores_per_node_redo.values()])

comet_path = '1-full_IO/results/trial_1/'
comet_redo1 = '1-full_IO/results/1_node_redo/1node_redo_1/'
comet_redo2 = '1-full_IO/results/1_node_redo/1node_redo_2/'

repeats = 3

In [3]:
def reduce_to_means(path, N, n):
    """Helper function to reduce data from numpy arrays.
    
    First it takes the mean across all ranks for each timing,
    then it takes the mean and standard deviation across the repeats.
    
    Parameters
    ----------
    n : int
        number of processes used in run
        
    Returns
    -------
    means : list
        mean across number of repeats for each timing
    stds : list
        standard deviation across repeats for each timing
        
    """

    
    init_total = np.empty(repeats)
    init_top = np.empty_like(init_total)
    init_traj = np.empty_like(init_total)
    open_traj = np.empty_like(init_total)
    n_atoms = np.empty_like(init_total)
    set_units = np.empty_like(init_total)
    io = np.empty_like(init_total)
    io_per_frame = np.empty_like(init_total)
    copy_data = np.empty_like(init_total)
    copy_box = np.empty_like(init_total)
    get_dataset = np.empty_like(init_total)
    set_ts_position = np.empty_like(init_total)
    convert_units = np.empty_like(init_total)
    rmsd = np.empty_like(init_total)
    rmsd_per_frame = np.empty_like(init_total)
    wait = np.empty_like(init_total)
    comm_gather = np.empty_like(init_total)
    total = np.empty_like(init_total)

    _dict = {f'a{i}': np.load(f'{path}/{N}node_{i}/{n}process_times.npy') for i in range(1, repeats+1)}

    for i, array in enumerate(_dict.values()):
        init_total[i] = np.mean(array[:, 1])
        init_top[i] = np.mean(array[:, 2])
        init_traj[i] = np.mean(array[:, 3])
        open_traj[i] = np.mean(array[:, 4])
        n_atoms[i] = np.mean(array[:, 5])
        set_units[i] = np.mean(array[:, 6])
        io[i] = np.mean(array[:, 7])
        io_per_frame[i] = np.mean(array[:, 8])
        copy_data[i] = np.mean(array[:, 9])
        copy_box[i] = np.mean(array[:, 10])
        get_dataset[i] = np.mean(array[:, 11])
        set_ts_position[i] = np.mean(array[:, 12])
        convert_units[i] = np.mean(array[:, 13])
        rmsd[i] = np.mean(array[:, 14])
        rmsd_per_frame[i] = np.mean(array[:, 15])
        wait[i] = np.mean(array[:, 16])
        comm_gather[i] = np.mean(array[:, 17])
        total[i] = np.mean(array[:, 18])

    means = [n, np.mean(init_total), np.mean(init_top), np.mean(init_traj), np.mean(open_traj), 
             np.mean(n_atoms), np.mean(set_units), np.mean(io), np.mean(io_per_frame),
             np.mean(copy_data), np.mean(copy_box), np.mean(get_dataset), np.mean(set_ts_position), np.mean(convert_units),
             np.mean(rmsd), np.mean(rmsd_per_frame), np.mean(wait), np.mean(comm_gather), np.mean(total)]
    stds = [n, np.std(init_total), np.std(init_top), np.std(init_traj), np.std(open_traj), 
             np.std(n_atoms), np.std(set_units), np.std(io), np.std(io_per_frame),
             np.std(copy_data), np.std(copy_box), np.std(get_dataset), np.std(set_ts_position), np.std(convert_units),
             np.std(rmsd), np.std(rmsd_per_frame), np.std(wait), np.std(comm_gather), np.std(total)]

    return means, stds

In [4]:
def all_process_dataframe(which_hpc):
    """Gives DataFrame of averaged timings for all N_process runs.
    
    Returns
    -------
    times_dframe : pd.DataFrame
        benchmark times with timings first averaged across all ranks, then averaged across repeats
    stds_dframe : pd.DataFrame 
        standard deviation of the timings when averaged across repeats   
    
    """
    columns = ['N_Processes', 'Total_Initialize', 'Initialize_Topology', 
                'Initialize_Trajectory', 'Open_Trajectory', 'Set_n_atoms',
                'Set_Units', 'IO', 'IO/Frame', 'Copy_Data', 'Copy_Box',
                'Get_Dataset', 'Set_ts_Position', 'Convert_Units',
                'RMSD', 'RMSD/Frame', 'Wait', 'Comm_Gather', 'Total_Benchmark_Time']
    
    count = -1
    
    if which_hpc == 'Redo1':
        data_buffer = np.empty(shape=(redo_sum, 19), dtype=float)
        stds_buffer = np.empty(shape=(redo_sum, 19), dtype=float)
        for N in [1]:
            for i, cores in enumerate(cores_per_node_redo[f'{N}'], count+1):
                count = i
                means, stds = reduce_to_means(comet_redo1, N, cores)
                for j in range(19):
                    data_buffer[i, j] = means[j]
                    stds_buffer[i, j] = stds[j]
                    
    elif which_hpc == 'Redo2':
        data_buffer = np.empty(shape=(redo_sum, 19), dtype=float)
        stds_buffer = np.empty(shape=(redo_sum, 19), dtype=float)
        for N in [1]:
            for i, cores in enumerate(cores_per_node_redo[f'{N}'], count+1):
                count = i
                means, stds = reduce_to_means(comet_redo2, N, cores)
                for j in range(19):
                    data_buffer[i, j] = means[j]
                    stds_buffer[i, j] = stds[j]
    
    elif which_hpc == 'Comet':
        data_buffer = np.empty(shape=(comet_sum, 19), dtype=float)
        stds_buffer = np.empty(shape=(comet_sum, 19), dtype=float)
        for N in [1,2,4,6,8,12,16,20,24,28,32]:
            for i, cores in enumerate(cores_per_node_comet[f'{N}'], count+1):
                count = i
                means, stds = reduce_to_means(comet_path, N, cores)
                for j in range(19):
                    data_buffer[i, j] = means[j]
                    stds_buffer[i, j] = stds[j]


    times_dframe = pd.DataFrame(list(data_buffer), columns=columns).set_index('N_Processes')
    stds_dframe = pd.DataFrame(list(stds_buffer), columns=columns).set_index('N_Processes')
            
    return times_dframe, stds_dframe

In [5]:
def get_raw_data(path, N, n, averaged=False):
    """Gets the raw data from all repeats and displays in a pandas DataFrame.
    
    If averaged=True, it takes the average and standard deviation across all 
    repeats for all (rank x timing) elements.
    
    Parameters
    ----------
    n : int
        number of processes used in run
    averaged: bool (optional)
    
    Returns
    -------
    all data (if averaged=False) : pd.DataFrame
        pandas dataframe of raw data arrays stacked horizontally with no reductions
    means (if averaged=True) : pd.DataFrame
        mean across repeats for each (rank x timing) element
    stds (if averaged=True) : pd.DataFrame 
        standard deviation across repeats for each (rank x timing) element
    
    """
    
    columns = ['rank', 'init_total', 'init_topology', 'init_trajectory', 
               'open_traj', 'n_atoms', 'set_units', 'io', 'io/frame',
               'copy_data', 'copy_box', 'get_dataset', 'set_ts_position', 'convert_units',
               'rmsd_compute', 'rmsd_compute/frame', 'wait', 'comm_gather',
               'total bench']
    
    _dict = {f'a{i}': np.load(f'{path}/{N}node_{i}/{n}process_times.npy') for i in range(1, repeats+1)}
    
    if averaged:
        means_buffer = np.zeros(shape=(n,19), dtype=float)
        stds_buffer = np.zeros(shape=(n,19), dtype=float)
        
        # fills in means and std arrays 1 element at a time
        for i in range(n):
            for j in range(19):
                temp_array = np.empty(repeats, dtype=float) 
                for trial, k in enumerate(range(len(temp_array)), 1):
                    temp_array[k] = _dict[f'a{trial}'][i, j]
                means_buffer[i, j] = np.mean(temp_array)
                stds_buffer[i, j] = np.std(temp_array)
          
        means = pd.DataFrame(list(means_buffer), columns=columns).set_index('rank')
        stds = pd.DataFrame(list(stds_buffer), columns=columns).set_index('rank')
        return means, stds
    
    arrays = tuple(_dict.values())
    a = np.hstack(arrays)

    return pd.DataFrame(list(a), columns=repeats*columns)

In [6]:
all_process_dataframe('Comet')[0]

,Total_Initialize,Initialize_Topology,Initialize_Trajectory,Open_Trajectory,Set_n_atoms,Set_Units,IO,IO/Frame,Copy_Data,Copy_Box,Get_Dataset,Set_ts_Position,Convert_Units,RMSD,RMSD/Frame,Wait,Comm_Gather,Total_Benchmark_Time
N_Processes,,,,,,,,,,,,,,,,,,
1.0,3.016406,2.084741,0.927625,0.261255,0.441998,0.001214,3379.290670,0.037506,100.863371,54.331279,3126.425424,10.118493,20.750307,17.272525,0.000192,0.000020,0.000345,3399.579967
4.0,3.213461,2.219818,0.989127,0.410941,0.335015,0.001308,1050.931556,0.046656,25.908511,14.315618,985.014902,2.778233,5.663392,4.499362,0.000200,0.001391,0.000389,1058.646158
8.0,3.228910,2.142712,1.081739,0.336268,0.272964,0.001333,553.830264,0.049175,12.898608,7.089198,520.494910,1.479848,2.998446,2.310702,0.000205,0.001055,0.000447,559.371379
16.0,2.946921,2.153326,0.789065,0.324917,0.341254,0.001301,616.384759,0.109457,8.042300,6.475174,594.669762,0.769360,1.530066,1.160392,0.000206,0.007713,0.000434,620.500219
24.0,3.331367,2.234403,1.092044,0.215075,0.480122,0.001300,671.602618,0.178894,5.481764,5.900900,654.911820,0.511829,0.996533,0.766487,0.000204,0.000729,0.000359,675.701559
24.0,12.326566,2.194590,10.126631,4.511873,5.553840,0.001389,428.564094,0.114155,4.940886,3.633208,414.838026,0.597048,1.064237,0.795059,0.000212,0.003782,0.001004,441.690504
36.0,3.181252,2.259335,0.915743,0.618302,0.121256,0.001388,37.646908,0.015042,3.035275,1.440493,29.557033,0.576804,0.793382,0.594516,0.000238,0.028455,0.001731,41.452861
48.0,2.876740,2.282752,0.588398,0.554140,0.012264,0.001346,34.115183,0.018174,2.433512,1.146909,27.439265,0.661396,0.627838,0.495331,0.000264,0.023831,0.002142,37.513227
72.0,9.017326,2.172415,6.839422,6.405845,0.284037,0.001411,235.722290,0.188338,1.553839,2.223073,230.226843,0.236599,0.372048,0.279553,0.000223,0.030460,0.001201,245.050830


In [7]:
all_process_dataframe('Redo1')[0]

,Total_Initialize,Initialize_Topology,Initialize_Trajectory,Open_Trajectory,Set_n_atoms,Set_Units,IO,IO/Frame,Copy_Data,Copy_Box,Get_Dataset,Set_ts_Position,Convert_Units,RMSD,RMSD/Frame,Wait,Comm_Gather,Total_Benchmark_Time
N_Processes,,,,,,,,,,,,,,,,,,
1.0,2.878515,2.059717,0.814929,0.223359,0.423028,0.001199,3337.296212,0.037040,99.166705,53.130326,3088.456105,10.107095,20.401424,17.231112,0.000191,0.000023,0.000444,3357.406306
4.0,5.554048,4.722963,0.826881,0.330007,0.298356,0.001292,1394.603653,0.061914,26.528180,15.614221,1326.531263,2.777309,5.599316,4.484059,0.000199,0.001093,0.001222,1404.644076
8.0,3.361284,2.144491,1.212511,0.443691,0.416147,0.001280,794.469895,0.070542,13.313186,7.805577,759.921961,1.477408,2.977495,2.293007,0.000204,0.000384,0.000253,800.124823
16.0,2.812409,2.157776,0.650104,0.249151,0.314971,0.001330,676.900615,0.120204,7.766857,6.363414,655.590664,0.768609,1.527147,1.159923,0.000206,0.000753,0.000511,680.874211
24.0,3.070144,2.204599,0.860672,0.217230,0.349004,0.001336,743.821756,0.198131,5.524686,6.170402,726.820785,0.507179,0.992364,0.766601,0.000204,0.005526,0.000431,747.664458


In [8]:
all_process_dataframe('Redo2')[0]

,Total_Initialize,Initialize_Topology,Initialize_Trajectory,Open_Trajectory,Set_n_atoms,Set_Units,IO,IO/Frame,Copy_Data,Copy_Box,Get_Dataset,Set_ts_Position,Convert_Units,RMSD,RMSD/Frame,Wait,Comm_Gather,Total_Benchmark_Time
N_Processes,,,,,,,,,,,,,,,,,,
1.0,2.717831,2.087029,0.626683,0.215232,0.268748,0.001175,3256.258399,0.036140,100.164613,54.072020,3004.240197,10.216690,20.993178,17.170450,0.000191,0.000051,0.000530,3276.147262
4.0,2.765551,2.182134,0.579157,0.170002,0.292644,0.001279,1078.492544,0.047880,26.040404,14.169972,1012.006075,2.816887,5.763683,4.533277,0.000201,0.003155,0.001406,1085.795934
8.0,2.795643,2.150109,0.641159,0.273025,0.167874,0.001293,758.087371,0.067311,13.356068,8.265359,723.035728,1.464559,3.007906,2.304397,0.000205,0.002021,0.000402,763.189835
16.0,2.858121,2.164030,0.689622,0.182024,0.357591,0.001347,531.077408,0.094309,7.448000,5.417764,511.088383,0.774073,1.536816,1.161910,0.000206,0.001271,0.000587,535.099296
24.0,2.992710,2.194703,0.793112,0.180184,0.339082,0.001368,757.068977,0.201660,5.603787,5.949843,739.918588,0.508282,0.997240,0.767422,0.000204,0.000989,0.000343,760.830440
